## Introduction
<p>Sports clothing and athleisure attire is a huge industry, worth approximately <a href="https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/">$193 billion in 2021</a> with a strong growth forecast over the next decade! </p>
<p>In this notebook, I will be a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. I will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  </p>
<p>The database provided, <code>sports</code>, contains five tables, with <code>product_id</code> being the primary key for all of them: </p>

## Tables

<h3 id="info"><code>info</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>description</code></td>
<td><code>varchar</code></td>
<td>Description of the product</td>
</tr>
</tbody>
</table>
<h3 id="finance"><code>finance</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>listing_price</code></td>
<td><code>float</code></td>
<td>Listing price for product</td>
</tr>
<tr>
<td><code>sale_price</code></td>
<td><code>float</code></td>
<td>Price of the product when on sale</td>
</tr>
<tr>
<td><code>discount</code></td>
<td><code>float</code></td>
<td>Discount, as a decimal, applied to the sale price</td>
</tr>
<tr>
<td><code>revenue</code></td>
<td><code>float</code></td>
<td>Amount of revenue generated by each product, in US dollars</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>rating</code></td>
<td><code>float</code></td>
<td>Product rating, scored from <code>1.0</code> to <code>5.0</code></td>
</tr>
<tr>
<td><code>reviews</code></td>
<td><code>float</code></td>
<td>Number of reviews for the product</td>
</tr>
</tbody>
</table>
<h3 id="traffic"><code>traffic</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>last_visited</code></td>
<td><code>timestamp</code></td>
<td>Date and time the product was last viewed on the website</td>
</tr>
</tbody>
</table>
<h3 id="brands"><code>brands</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>brand</code></td>
<td><code>varchar</code></td>
<td>Brand of the product</td>
</tr>
</tbody>
</table>


## 1. Counting missing values
<p>We will be dealing with missing data as well as numeric, string, and timestamp data types to draw insights about the products in the online store. Let's start by finding out how complete the data is.</p>

In [22]:
%%sql
postgresql:///sports

SELECT COUNT(*) AS total_rows,
COUNT(description) AS count_description,
COUNT(listing_price) AS count_listing_price,
COUNT(last_visited) AS count_last_visited
FROM info 
    INNER JOIN finance 
    ON finance.product_id = info.product_id
    INNER JOIN traffic 
    ON traffic.product_id = info.product_id;
    

1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


#another_way 

SELECT COUNT(*) AS total_rows,

COUNT(*) - COUNT(description) AS missing_description,

COUNT(*) - COUNT(listing_price) AS missing_listing_price,

COUNT(*) - COUNT(last_visited) AS missing_last_visited

    FROM info 

    INNER JOIN finance 

    ON finance.product_id = info.product_id

    INNER JOIN traffic 

    ON traffic.product_id = info.product_id;
    

<p>We can see the database contains 3,179 products in total. Of the columns we previewed, only one &mdash; <code>last_visited</code> &mdash; is missing more than five percent of its values. Now let's turn our attention to pricing. </p>

## 2. Nike vs Adidas pricing

<p>How do the price points of Nike and Adidas products differ?  We will run a query to produce a distribution of the <code>listing_price</code> and the count for each price, grouped by <code>brand</code>. </p>

In [ ]:
%%sql

SELECT brand,CAST(listing_price AS INTEGER),COUNT(*)
FROM finance 
INNER JOIN brands 
USING(product_id)
WHERE listing_price > 0
GROUP BY brand,listing_price
ORDER BY listing_price DESC;

<p>It turns out there are 77 unique prices for the products in our database, which makes the output of our last query quite difficult to analyze. </p>

## 3. Labeling price ranges

<p>Let's build on our previous query by assigning labels to different price ranges, grouping by <code>brand</code> and <code>label</code>. We will also include the total <code>revenue</code> for each price range and <code>brand</code>. </p>

In [26]:
%%sql


SELECT b.brand, COUNT(f.*), SUM(f.revenue) as total_revenue,
CASE WHEN f.listing_price < 42 THEN 'Budget'
    WHEN f.listing_price >= 42 AND f.listing_price < 74 THEN 'Average'
    WHEN f.listing_price >= 74 AND f.listing_price < 129 THEN 'Expensive'
    ELSE 'Elite' END AS price_category
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


<p>Interestingly, grouping products by brand and price range allows us to see that Adidas items generate more total revenue regardless of price category! Specifically, <code>"Elite"</code> Adidas products priced \$129 or more typically generate the highest revenue, so the company can potentially increase revenue by shifting their stock to have a larger proportion of these products!</p>

## 4. Average discount by brand

<p>Note we have been looking at <code>listing_price</code> so far. The <code>listing_price</code> may not be the price that the product is ultimately sold for. To understand <code>revenue</code> better, let's take a look at the <code>discount</code>. We would like to know whether there is a difference in the amount of <code>discount</code> offered between brands, as this could be influencing <code>revenue</code>.</p>

In [28]:
%%sql
    
SELECT b.brand, AVG(f.discount) * 100 AS average_discount
FROM brands AS b
INNER JOIN finance AS f 
    ON b.product_id = f.product_id
GROUP BY b.brand
HAVING b.brand IS NOT NULL
ORDER BY average_discount;

 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


<p>Strangely, no <code>discount</code> is offered on Nike products! In comparison, not only do Adidas products generate the most revenue, but these products are also heavily discounted! </p>
<p>CONCLUSION : To improve revenue further, the company could try to reduce the amount of discount offered on Adidas products, and monitor sales volume to see if it remains stable. Alternatively, it could try offering a small discount on Nike products. This would reduce average revenue for these products, but may increase revenue overall if there is an increase in the volume of Nike products sold. </p>

## 5. Correlation between revenue and reviews


<p>Now explore whether relationships exist between the columns in our database. We will check the strength and direction of a correlation between <code>revenue</code> and <code>reviews</code>. </p>

In [30]:
%%sql

SELECT corr(r.reviews,f.revenue) AS review_revenue_corr
FROM reviews AS r
INNER JOIN finance AS f
ON f.product_id = r.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.6518512283481301


<p>Interestingly, there is a medium to strong positive correlation between <code>revenue</code> and <code>reviews</code>. This means, potentially, if we can get more reviews on the company's website, it may increase sales of those items with a larger number of reviews. </p>

## 6. Ratings and reviews by product description length

<p>Perhaps the length of a product's <code>description</code> might influence a product's <code>rating</code> and <code>reviews</code> &mdash; if so, the company can produce content guidelines for listing products on their website and test if this influences <code>revenue</code>. Let's check this out!</p>

In [32]:
%%sql

SELECT TRUNC(LENGTH(description),-2) AS description_length,
ROUND(AVG(CAST(rating AS numeric)),2) AS average_rating
FROM reviews 
INER JOIN info
USING (product_id)
WHERE description IS NOT NULL
GROUP BY description_length 
ORDER BY description_length;

 * postgresql:///sports
7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


Unfortunately, there doesn't appear to be a clear pattern between the length of a product's description and its rating.

As we know a correlation exists between reviews and revenue, one approach the company could take is to run experiments with different sales processes encouraging more reviews from customers about their purchases, such as by offering a small discount on future purchases. 

## 7. Reviews by month and brand


<p>Let's take a look at the volume of <code>reviews</code> by month to see if there are any trends or gaps we can look to exploit.</p>

In [34]:
%%sql
SELECT brand,date_part('month',last_visited) AS month,COUNT(reviews.*) AS num_reviews
FROM traffic
INNER JOIN reviews
ON reviews.product_id = traffic.product_id
INNER JOIN brands
ON brands.product_id = traffic.product_id
GROUP BY brand,month
HAVING brand IS NOT NULL AND date_part('month',last_visited) IS NOT NULL
ORDER BY brand,month;


 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1.0,253
Adidas,2.0,272
Adidas,3.0,269
Adidas,4.0,180
Adidas,5.0,172
Adidas,6.0,159
Adidas,7.0,170
Adidas,8.0,189
Adidas,9.0,181
Adidas,10.0,192


Conclusion: Looks like product reviews are highest in the first quarter of the calendar year, so there is scope to run experiments aiming to increase the volume of reviews in the other nine months!

## 8. Footwear product performance

<p>So far, we have been primarily analyzing Adidas vs Nike products. Now, let's switch our attention to the type of products being sold. As there are no labels for product type, we will create a Common Table Expression (CTE) that filters <code>description</code> for keywords, then use the results to find out how much of the company's stock consists of footwear products and the median <code>revenue</code> generated by these items.</p>

In [36]:
%%sql
WITH footwear AS(SELECT description,revenue
FROM info
INNER JOIN finance
USING(product_id)
WHERE description ILIKE '%shoe%' 
                 OR description ILIKE'%trainer%' 
                 OR description ILIKE '%foot%'
                 AND description IS NOT NULL)
SELECT COUNT(footwear.*) AS num_footwear_products,PERCENTILE_disc(0.5) WITHIN GROUP(ORDER BY revenue) AS median_footwear_revenue
FROM footwear


 * postgresql:///sports
1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


Conclusion: recall from the first task that we found there are 3,117 products without missing values for description. Of those, 2,700 are footwear products, which accounts for around 85% of the company's stock. They also generate a median revenue of over $3000 dollars!

## 9. Clothing product performance

<p>This is interesting, but we have no point of reference for whether footwear's <code>median_revenue</code> is good or bad compared to other products. So let's examine how this differs to other clothing products. I will add a filter to count the number of products and <code>median_revenue</code> of products that are not in <code>footwear</code> instead of making a new long query.</p>

In [ ]:
%%sql

WITH footwear AS(SELECT description,revenue
FROM info
INNER JOIN finance
USING(product_id)
WHERE (description ILIKE '%shoe%' OR description ILIKE'%trainer%' OR description ILIKE '%foot%')
                 AND (description IS NOT NULL))
SELECT COUNT(INFO.*) AS num_clothing_products,
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY revenue) AS median_clothing_revenue
FROM info
INNER JOIN finance
USING(product_id)
WHERE description NOT IN (SELECT description FROM footwear);